# Outlier Detection (Movies)
In this notebook, we compute Local Outlier Factor scores for each movie represented by its popular features to detect outliers. LOF is a "normalized distance-based approach where the normalization factor corresponds to the average local data density." [Data Mining Textbook]

## Local Outlier Factor
Intuitively, we find that movies with unusual feature combinations, namely unusual cast ensembles or surprising individual casting choices for the genre, are given the highest LOF score, thus indicating the highest "outlier" rating.
- The Royal Tenenbaums
- Indecent Proposal
- Mars Attacks

## Isolation Forest
Similar to DBSCAN and LOF, we find that feature-rich movies with exceptionally popular "superstar" cast are detected as outliers by Isolation Forest. In particular, the biggest outliers have many A-star actors from a wide variety of genres, with some surprising casting choices given the movie genres.
- How The West Was Won
- Predator 2
- Antz
- Sleepy Hollow
- Batman Returns

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import gzip
import json

import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

# Enable notebook renderer for Altair
alt.renderers.enable('default')

DATA_PATH = "../data"
NETFLIX_FOLDER_PATH = os.path.join(DATA_PATH, "netflix_prize")
IMDB_FOLDER_PATH = os.path.join(DATA_PATH, "imdb")
MIN_OCCURRENCES = 20

## Load Data

In [ ]:
# Load movie features
MOVIE_FEATURES_PATH = os.path.join(DATA_PATH, f"processed/movie_features_{MIN_OCCURRENCES}.pickle")
with open(MOVIE_FEATURES_PATH, "rb") as f:
    movie_features = pickle.load(f)

# Load feature mapping
FEATURE_MAPPING_PATH = os.path.join(DATA_PATH, f"processed/feature_mapping_{MIN_OCCURRENCES}.pickle")
with open(FEATURE_MAPPING_PATH, "rb") as f:
    feature_mapping = pickle.load(f)

feature_to_id = feature_mapping['feature_to_id']
id_to_feature = feature_mapping['id_to_feature']

# Number of features
num_features = len(feature_to_id)
print(f"Number of features: {num_features}")

### Create Movie Feature Matrix

In [ ]:
# List of movie IDs and feature IDs
movie_ids = list(movie_features.keys())
feature_ids = list(id_to_feature.keys())

# Create an empty DataFrame
movie_feature_matrix = pd.DataFrame(0, index=movie_ids, columns=feature_ids)

# Fill the DataFrame
for movie_id, features in movie_features.items():
    movie_feature_matrix.loc[movie_id, features] = 1

print(f"Movie feature matrix shape: {movie_feature_matrix.shape}")

In [4]:
# Load Netflix to IMDb mapping
NETFLIX_TO_IMDB_PATH = os.path.join(DATA_PATH, "netflix_to_imdb.json")
with open(NETFLIX_TO_IMDB_PATH, "r") as f:
    netflix_to_imdb = json.load(f)

# Create a mapping from Netflix movie IDs to IMDb IDs
netflix_ids = set(movie_ids)
netflix_to_imdb_filtered = {nid: imdb_id for nid, imdb_id in netflix_to_imdb.items() if nid in netflix_ids}

In [ ]:
# Load IMDb title.basics.tsv.gz
TITLE_BASICS_PATH = os.path.join(IMDB_FOLDER_PATH, "title.basics.tsv.gz")

imdb_titles = {}

with gzip.open(TITLE_BASICS_PATH, 'rt', encoding='utf-8') as f:
    # Skip header
    next(f)
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) != 9:
            continue
        tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres = parts
        imdb_titles[tconst] = primaryTitle

print(f"Loaded {len(imdb_titles)} IMDb titles.")

In [6]:
# Map Netflix movie IDs to titles
movie_titles = {}

for netflix_id in movie_ids:
    imdb_id = netflix_to_imdb_filtered.get(netflix_id)
    if imdb_id and imdb_id in imdb_titles:
        movie_titles[netflix_id] = imdb_titles[imdb_id]
    else:
        movie_titles[netflix_id] = f"Unknown Title ({netflix_id})"

# Add movie titles to the DataFrame
movie_feature_matrix['title'] = movie_feature_matrix.index.map(movie_titles)

In [7]:
# Create a list of feature names for each movie
def get_feature_names(feature_ids):
    return [id_to_feature[feat_id] for feat_id in feature_ids]

movie_feature_matrix['features'] = movie_feature_matrix.index.map(
    lambda x: ', '.join(get_feature_names(movie_features.get(x, [])))
)

# Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Drop the 'title' and 'features' columns to focus on numeric data
feature_data = movie_feature_matrix.drop(columns=['title', 'features'])
feature_data_scaled = feature_data

# Initialize the Local Outlier Factor (LOF) model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)  # 5% outliers

# Fit the model and predict outlier scores
lof_scores = -lof.fit_predict(feature_data)
# The negative values indicate how strong the outlier effect is (higher means more outlier-like)

# Add LOF scores to the original movie feature matrix
movie_feature_matrix['LOF_Score'] = lof.negative_outlier_factor_

# Sort movies by their LOF score (outliers at the top)
outliers = movie_feature_matrix.sort_values(by='LOF_Score', ascending=True)

# Display the top potential outliers
outliers.head(20)[['title', 'features', 'LOF_Score']]

In [ ]:
# Prepare data for plotting
plot_data = movie_feature_matrix.copy()

# Reset index and ensure column names are strings for Altair compatibility
plot_data = plot_data.reset_index(drop=True)
plot_data.columns = plot_data.columns.map(str)  # Ensure all column names are strings

# Add an 'is_outlier' column to mark outliers
plot_data['is_outlier'] = plot_data['LOF_Score'] < -2e+10  # Mark outliers

# Identify outliers and inliers
outliers = plot_data[plot_data['is_outlier']]
inliers = plot_data[~plot_data['is_outlier']]

# Calculate the number of inliers to sample to keep total rows <= 5000
num_outliers = len(outliers)
num_inliers_to_sample = max(5000 - num_outliers, 0)

# Sample inliers
inliers_sampled = inliers.sample(n=num_inliers_to_sample, random_state=42)

# Combine outliers and sampled inliers
plot_data_sampled = pd.concat([outliers, inliers_sampled], ignore_index=True)

# Prepare feature data for PCA (excluding non-numeric columns)
numeric_columns = feature_data.columns.map(str)
numeric_data = plot_data_sampled[numeric_columns]

# Apply PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(numeric_data)

# Add PCA components to the DataFrame
plot_data_sampled['PC1'] = pca_results[:, 0]
plot_data_sampled['PC2'] = pca_results[:, 1]

# Create a selection object for zooming and panning
brush = alt.selection_interval()

# Create scatter plot with PCA components, zoom, and tooltips
chart = alt.Chart(plot_data_sampled).mark_circle(size=60).encode(
    x=alt.X('PC1:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('PC2:Q', scale=alt.Scale(zero=False)),
    color=alt.condition(
        alt.datum.is_outlier,
        alt.value('red'),  # Outliers in red
        alt.value('rgba(0,0,255,0.1)')  # Inliers in semi-transparent blue
    ),
    tooltip=[alt.Tooltip('title:N'), alt.Tooltip('LOF_Score:Q'), alt.Tooltip('features:N')]
).add_selection(
    brush
).properties(
    width=800,
    height=400,
    title="LOF Outliers (Sampled Data)"
).interactive()

chart

# Isolation Forest

In [ ]:
# Isolation Forest
from sklearn.ensemble import IsolationForest

# Prepare the data by dropping non-numeric columns
feature_data = movie_feature_matrix.drop(columns=['title', 'features', 'LOF_Score'], errors='ignore')

# Initialize the Isolation Forest model
iso_forest = IsolationForest(
    n_estimators=100,       # Number of base estimators
    contamination=0.05,     # Assume 5% of the data are outliers
    random_state=42
)

# Fit the model
iso_forest.fit(feature_data)

# Get anomaly scores (the lower, the more abnormal)
anomaly_scores = iso_forest.decision_function(feature_data)

# Predict outliers (-1 for anomalies, 1 for normal instances)
outlier_preds = iso_forest.predict(feature_data)

# Add scores to the DataFrame
movie_feature_matrix['Isolation_Score'] = anomaly_scores
movie_feature_matrix['Isolation_Outlier'] = outlier_preds

# Sort movies by their anomaly scores (outliers at the top)
iso_outliers = movie_feature_matrix.sort_values(by='Isolation_Score')

# Display the top potential outliers
iso_outliers.head(20)[['title', 'features', 'Isolation_Score']]

In [ ]:
# Prepare data for plotting
plot_data_iso = movie_feature_matrix.copy()

# Reset index and ensure column names are strings for Altair compatibility
plot_data_iso = plot_data_iso.reset_index(drop=True)
plot_data_iso.columns = plot_data_iso.columns.map(str)

# Add an 'is_outlier' column to mark outliers
plot_data_iso['is_outlier'] = plot_data_iso['Isolation_Outlier'] == -1  # Mark outliers

# Identify outliers and inliers
outliers_iso = plot_data_iso[plot_data_iso['is_outlier']]
inliers_iso = plot_data_iso[~plot_data_iso['is_outlier']]

# Calculate the number of inliers to sample to keep total rows <= 5000
num_outliers_iso = len(outliers_iso)
num_inliers_to_sample_iso = max(5000 - num_outliers_iso, 0)

# Sample inliers
inliers_sampled_iso = inliers_iso.sample(n=num_inliers_to_sample_iso, random_state=42)

# Combine outliers and sampled inliers
plot_data_sampled_iso = pd.concat([outliers_iso, inliers_sampled_iso], ignore_index=True)

# Prepare feature data for PCA (excluding non-numeric columns)
numeric_columns = feature_data.columns.map(str)
numeric_data_iso = plot_data_sampled_iso[numeric_columns]

# Apply PCA
pca_iso = PCA(n_components=2)
pca_results_iso = pca_iso.fit_transform(numeric_data_iso)

# Add PCA components to the DataFrame
plot_data_sampled_iso['PC1'] = pca_results_iso[:, 0]
plot_data_sampled_iso['PC2'] = pca_results_iso[:, 1]

# Create a selection object for zooming and panning
brush_iso = alt.selection_interval()

# Create scatter plot with PCA components, zoom, and tooltips
chart_iso = alt.Chart(plot_data_sampled_iso).mark_circle(size=60).encode(
    x=alt.X('PC1:Q', scale=alt.Scale(zero=False)),
    y=alt.Y('PC2:Q', scale=alt.Scale(zero=False)),
    color=alt.condition(
        alt.datum.is_outlier,
        alt.value('red'),  # Outliers in red
        alt.value('rgba(0,0,255,0.1)')  # Inliers in semi-transparent blue
    ),
    tooltip=[alt.Tooltip('title:N'), alt.Tooltip('Isolation_Score:Q'), alt.Tooltip('features:N')]
).add_selection(
    brush_iso
).properties(
    width=800,
    height=400,
    title="Isolation Forest Outliers (Sampled Data)"
).interactive()

chart_iso